# Суммаризатор видео

## 0. Устанавливаем необходимые зависимости

In [1]:
!pip install yt-dlp moviepy ffmpeg-python transformers[sentencepiece] faster-whisper torch

## 1. Скачиваем видео по ссылке

In [2]:
from pathlib import Path
import yt_dlp as ytdl

def download_video(url: str, out_dir: str = "downloads") -> Path:
    out_dir_p = Path(out_dir)
    out_dir_p.mkdir(parents=True, exist_ok=True)
    ydl_opts = {
        'outtmpl': str(out_dir_p / '%(title)s.%(ext)s'),
        'format': 'bestvideo+bestaudio/best',
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
    }
    with ytdl.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        # инфа содержит 'title' и 'ext'
        filename = ydl.prepare_filename(info)
    return Path(filename)

In [3]:
import os
os.environ['PATH'] += r';D:\ffmpeg\bin'
download_video('https://www.youtube.com/shorts/n5M0lxa_Zys')

WindowsPath('downloads/Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.webm')

## 2. Извлечение аудио

In [4]:
from moviepy import VideoFileClip
import ffmpeg

def extract_audio_to_wav(video_path: Path, out_wav: str = None, sr: int = 16000) -> Path:
    video_path = Path(video_path)
    if out_wav is None:
        out_wav = video_path.with_suffix('.wav')
    out_wav = Path(out_wav)

    clip = VideoFileClip(str(video_path))
    tmp_audio = video_path.with_suffix('.temp_audio.wav')
    clip.audio.write_audiofile(str(tmp_audio), fps=sr)
    clip.close()

    stream = ffmpeg.input(str(tmp_audio))
    stream = ffmpeg.output(stream, str(out_wav), ac=1, ar=sr)
    ffmpeg.run(stream, overwrite_output=True, quiet=True)

    try:
        os.remove(tmp_audio)
    except Exception:
        pass

    return out_wav

In [5]:
extract_audio_to_wav('downloads/Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.webm')

MoviePy - Writing audio in downloads\Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.temp_audio.wav


MoviePy - Done.


WindowsPath('downloads/Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.wav')

## 3. Траскрибация аудио

In [6]:
from faster_whisper import WhisperModel

def transcribe_audio_wisper(audio_path: Path, model_size: str = 'small', language: str  = 'ru', task: str = 'transcribe') -> str:
    model = WhisperModel(model_size, device='cpu', compute_type='float32')
    segments, info = model.transcribe(str(audio_path), beam_size=5, language=language, task=task)

    texts = []
    for segment in segments:
        texts.append(segment.text)
    full_text = ' '.join(texts)
    return full_text

C:\Users\Jize\AppData\Local\Programs\Python\Python313\Lib\site-packages\ctranslate2\__init__.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [7]:
text = transcribe_audio_wisper('downloads/Пример красоты математики ⧸ Голосовое сообщение ⧸ Савватеев⧸ Мацан #математика #shorts.wav')

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

C:\Users\Jize\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jize\.cache\huggingface\hub\models--Systran--faster-whisper-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
text

' Дайте пример, красоты и математики.  Ты видел построение пятиугольника циркулем и линейкой?  Да.  Видел, да?  То есть вот этот треугольник берется 72-72-36,  проводится без сектриса, вот здесь равнобедленный,  вот здесь равнобедленный подобно исходному,  вот это равно этому, равно этому,  после чего это все обозначается за х,  возникает золотое сечение в отношении вот этих двух сторон.  Мы его строим циркулем и линейкой и строим пятиугольник.  Ну, по-моему, это божественно.  Вот честно, то есть я не знаю,  я когда показал эту тему Лебедеву,  а он говорил мне, что золотое сечение — это переоцененный объект,  вообще, оно мне вообще не нужно, это все это...  Много раз вообще это в интернете говорил из пользы всякую веселую лексику.  Вот, и вот когда мы с ним встречались,  у нас был с ним коллап,  который тоже такой довольно популярный уже под миллион просмотров,  и я могу сейчас тебе покажу, как это красиво на самом деле.  И он признал, что это действительно красиво.  То есть это божест

## 4. Суммаризация текста

In [9]:
def chunk_text(text: str, max_chars: int = 3000):
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunks.append(text[start:end])
        start = end
    return chunks

In [10]:
from transformers import pipeline

def summarize_text(text: str, model_name: str, max_chunk_chars: int = 3000) -> str:
    summarizer = pipeline('summarization', model=model_name, truncation=True)
    chunks = chunk_text(text, max_chars=max_chunk_chars)
    summaries = []
    for i, ch in enumerate(chunks):
        res = summarizer(ch, max_length=200, min_length=30, do_sample=False)
        summaries.append(res[0]['summary_text'])

    if len(summaries) > 1:
        joined = ' '.join(summaries)
        res2 = summarizer(joined, max_length=250, min_length=50, do_sample=False)
        return res2[0]['summary_text']
    else:
        return summaries[0]

In [11]:
summarize_text(text, '2KKLabs/Lacia_sum_small_v1')

config.json: 0.00B [00:00, ?B/s]

C:\Users\Jize\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jize\.cache\huggingface\hub\models--2KKLabs--Lacia_sum_small_v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'**Необходимость золотого сечения в отношении вот этих двух сторон заключается в том, что они строим циркулем и линейкой.** Этот треугольник берется 72-72-36, который проводится без сектриса, и здесь равнобедлен, что приводит к золотому сечению. Лебедев говорил, что золотое сечение — это переоцененный объект, поэтому ему это не нужно. Он также был популярен под миллион просмотров, и он признал, что это божественная вещь.'

In [12]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 
tokenizer = GPT2Tokenizer.from_pretrained('RussianNLP/FRED-T5-Summarizer',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('RussianNLP/FRED-T5-Summarizer')
device='cpu'
model.to(device)

#input_text='<LM> Сократи текст.\n В деревне, затерянной среди зелёных холмов, жил старик по имени Иван. Его жизнь протекала медленно и размеренно. Каждое утро Иван выходил на поля, чтобы заботиться о своём скромном участке земли. Он выращивал картофель и морковь, которые были его главным источником пищи. Вечера старик проводил у камина, читая книги и вспоминая молодость. Жизнь в деревне была тяжёлая, но Иван находил в ней простые радости.'
input_ids=torch.tensor([tokenizer.encode(text)]).to(device)
outputs=model.generate(input_ids,eos_token_id=tokenizer.eos_token_id,
                    num_beams=5,
                    min_new_tokens=17,
                    max_new_tokens=200,
                    do_sample=True,
                    no_repeat_ngram_size=4,
                    top_p=0.9)
print(tokenizer.decode(outputs[0][1:]))

tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\Jize\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jize\.cache\huggingface\hub\models--RussianNLP--FRED-T5-Summarizer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

Построение пятиугольника циркулем и линейкой с помощью золотого сечения — это божественная вещь.</s>


In [13]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-absum")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-absum")

inputs = tokenizer(text, return_tensors='pt')
with torch.no_grad():
    hypotheses = model.generate(
        **inputs, 
        do_sample=True, top_p=0.95, num_return_sequences=10, 
        repetition_penalty=2.5,
        max_length=32,
    )
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

C:\Users\Jize\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jize\.cache\huggingface\hub\models--cointegrated--rut5-base-absum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

Я мог сейчас тебе показать в онлайне, как это красиво на самом деле. Давайте примера красоты
Это идеальное место в нашем доме.
Сегодня мы покажу, как это красиво. Знаем на самом деле
Что-то похоже на наши ворота. Но чем?
Принесите пример, лучшее построение семиугольника. Скажу вам свою идеальную цену... Потому что это божественно на
Он сделал мне вид, что это божественно на самом деле. И вы говорили моему серьезному специалисту в своем
В качестве примера, как построения пятиугольников?
Получила идею: “Я могу сейчас покажу, как это красиво на самом деле»
Это прекрасно на самом деле. И по-моему, это божественно
Не знаю, что в этой статье он был так популярен. Был с ней коллап
